In [1]:
import numpy as np
import re
import pymorphy2
from scipy.spatial import distance
from matplotlib import pylab as plt
import xlrd
import re
import json
import pprint
import urllib.request

ModuleNotFoundError: No module named 'pymorphy2'

In [2]:
# "чистка" строки от html-тегов
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

In [3]:
# требования по вакансиям с РосТруда
def read_from_rtrud(pname, ccode): #pname - навание вакансии, ccode - код города
    pname = pname.encode("utf-8")
    pname = urllib.request.quote(pname)

    with urllib.request.urlopen(
            "http://opendata.trudvsem.ru/api/v1/vacancies/region/{}00000000000?text={}"
                    .format(ccode, pname)) as url:
        s = url.read()
        s = s.decode('utf-8')


    # перевод из json в python-объект
    python_obj = json.loads(s)

    condition = True
    count = 0
    complist = []
    
    if 'vacancies' not in python_obj['results']:
        return complist
    
    for key in python_obj['results']['vacancies']:
        comp = key['vacancy']['duty']
        comp = cleanhtml(comp)
        comp = comp.split(';')
        # фильтрует элементы по длине
        for value in comp:
            if len(value) <= 6:
                continue
            else:
                value = ''.join(value)
                value = value.strip()
                if len(value) <=6:
                    continue
                complist.append(value)
                # print(value)
                # print('\n')
    return complist

In [4]:
# парсинг компетенций из вакансии по URL
def parse_comp(vacurl):
    with urllib.request.urlopen(
            vacurl) as url:
        s = url.read()
        s = s.decode('utf-8')
        s = cleanhtml(s)
    complist = re.findall('Требования:.*?Условия:', s)
    # убираем слова "Требования" и "Условия" и распиливаем строку в список
    # по одному URL несколько вакансий, поэтому здесь цикл
    comps = []
    for comp in complist:
        comp = comp.replace('Требования:','')
        comp = comp.replace('Условия:','')
        comp = comp.split(';')
        comps += comp
    return comps

In [5]:
# требования по вакансиям с HH
def read_from_hh(pname, ccode): #pname - навание вакансии, ccode - код города (на hh своя нумерация городов)
    pname = pname.encode("utf-8")
    pname = urllib.request.quote(pname)

    with urllib.request.urlopen(
            "https://api.hh.ru/vacancies/?text={}&area={}"
                    .format(pname, ccode)) as url:
        s = url.read()
        s = s.decode('utf-8')


    # перевод из json в python-объект
    python_obj = json.loads(s)

    condition = True
    count = 0
    complist = []

    for key in python_obj['items']:
        comp = parse_comp(key['url'])
        # фильтрует элементы по длине
        for value in comp:
            if len(value) <= 6:
                continue
            else:
                value = ''.join(value)
                value = value.strip()
                if len(value) <=6:
                    continue
                complist.append(value)
                # print(value)
                # print('\n')
    return complist

In [7]:
# сплошной текст разбивается на отдельные строки, а каждая строка - на отдельные слова, приведенные к нормальной форме
def AnalyText(text_list):
    result = []
    word_dict = {}
    obshc_word = []
    N=0
    morph = pymorphy2.MorphAnalyzer()
    word_in_str = []
    
    for line in text_list:
        d=re.split('\W+', line.lower())       
        for w in d:
            if len(w) > 2:
                asd=morph.parse(w)[0].normal_form
                word_dict[asd] = word_dict.get(asd,0) + 1
                word_in_str.append(asd)
                N += 1
        result.append(list(word_in_str))
        word_in_str.clear()
        
        
    # формирование списка слов, которые встречаются в тексте наиболее часто
    for key, value in word_dict.items():
        if value*1000/N > 8:
            obshc_word.append(key)
    
    return result, obshc_word

In [8]:
# "чистка" пустых элементов списка
def cler_space(res):
    for strin in res:
        while '' in strin:
            strin.remove('')
    return res

In [9]:
# удаление слов, которые встречаются в тексте наиболее часто
def delete_obshc(list_comp, word_obshc): # list_comp - список компетенций, разбитый по словам; word_obshc - список слов для удаления
    result = []
    for string in list_comp:
        result.append(list(set(string).difference(set(word_obshc))))
    return result

In [10]:
# формирование матрциы с количеством общих слов для каждой пары компетенций Костромская область - Лидер
def interseption(res1, res2):
    result = []
    for competen1 in res1:
        result_string = []
        for competen2 in res2:
            result_string.append(len(set.intersection(set(competen1), set(competen2))))
        result.append(result_string)
    return result

In [11]:
# формирование матрицы соответствия с учетом длин строк
def analitic_transform(analitic_massive, res_word_KO, res_word_lid):
    for i, string in enumerate(analitic_massive):
        for j, value in  enumerate(string):
            #print(len(res_word_KO[j]), 'index', j)
            #print(len(res_word_KO[i]), 'index', i)
            #print()
            if len(res_word_KO[j]) == 0 or len(res_word_lid[i]) == 0:
                analitic_massive[i][j] = 0
            else:
                #print(len(res_word_KO[j]))
                #print(len(res_word_lid[i]))
                #print()
                analitic_massive[i][j] = 1.*(value/len(res_word_KO[j]))*(value/len(res_word_lid[i]))
    return analitic_massive

In [12]:
# форомирование списков индексов компетенций для 3 групп
# 1 - Только лидерские компетенции
# 2 - Только костромские компетенции
# 3 - Компетенции, характерные для обеих групп
def start_split(analitic_massive, res_KO, res_lid):
    competention_lid = []
    competention_KO = []
    competention_inters = {}
    index_KO = set(range(0, len(res_KO), 1))
    for index, string_value in enumerate(analitic_massive):
        index_max_list = []
        if max(string_value)>0.15:
            max_value = max(string_value)
        else:
            max_value = 1
        for index_ko, value in enumerate(string_value):
            if value == max_value:
                index_max_list.append(index_ko)
        if len(index_max_list)==0:
            competention_lid.append(index)
        else:
            competention_inters[index] = index_max_list
    for list_values in competention_inters.values():
        competention_KO = index_KO.difference(list_values)
    return competention_lid, competention_KO, competention_inters

In [16]:
# получение компетенций Костромской области с ресурсов РосТруд и HeadHunter
text_KO_R_Trud = read_from_rtrud('Оператор станков с программным управлением', '44')
text_KO_HH = read_from_hh('Оператор станков с программным управлением', '1771') # 1771 - Костромская область
text_KO = list(set.union(set(text_KO_R_Trud), set(text_KO_HH)))

# получение компетенций лидеров с ресурсов РосТруд и HeadHunter
text_lid_R_Trud =  read_from_rtrud('Оператор станков с программным управлением', '50') # 50 - Московская область
text_lid_HH = read_from_hh('Оператор станков с программным управлением', '1') # 1 - Московская область
text_lid = list(set.union(set(text_lid_R_Trud), set(text_lid_HH)))



NameError: name 'urllib' is not defined

In [17]:
new_text_lid = []
for num, text in enumerate(text_lid):
    text_lid[num] = text.replace('и т.п.', '.')
    text_lid[num] = text.replace(' - ', '.')
for text in text_lid:
    comp = text.split('.')
    new_text_lid += comp
text_lid = new_text_lid

NameError: name 'text_lid' is not defined

In [18]:
res_KO_no_update = text_KO
res_lid_no_update = text_lid

NameError: name 'text_KO' is not defined

In [19]:
def program_analiz(text_KO, text_lid):
    
    #Чтение файлов
    #res_KO, res_KO_no_update = AnalyText_from_excel(text_KO)
    #res_lid, res_lid_no_update = AnalyText_from_excel(text_lid)
    
    res_KO, obshc_word_KO = AnalyText(text_KO)
    res_lid, obshc_word_lid = AnalyText(text_lid)
    

    obshc_word = set.union(set(obshc_word_KO), set(obshc_word_lid))
    
    print(obshc_word)
    
    #Убрать  ''
    res_KO = cler_space(res_KO)
    res_lid = cler_space(res_lid)
    

    
    res_KO = delete_obshc(res_KO, obshc_word)
    res_lid = delete_obshc(res_lid, obshc_word)
    
    
    #res_KO = norm_word(res_KO)
    #res_lid = norm_word(res_lid)
    
    analitic_massive = interseption(res_lid, res_KO)
    
    
    probability_analitic_massive = analitic_transform(analitic_massive, res_KO, res_lid)
        
    competention_lid, competention_KO, competention_inters = start_split(probability_analitic_massive, res_KO, res_lid)
    
    return competention_lid, competention_KO, competention_inters

In [20]:
competention_lid, competention_KO, competention_inters = program_analiz(list(text_KO), list(text_lid))

NameError: name 'text_KO' is not defined

In [21]:
competention_lid

NameError: name 'competention_lid' is not defined

In [13]:
competention_KO

NameError: name 'competention_KO' is not defined

In [14]:
competention_inters

NameError: name 'competention_inters' is not defined

In [15]:
# вывод компетенций, характерных для обеих групп
for key, value in competention_inters.items():
    print(res_lid_no_update[key].upper())
    for el in value:
        print(res_KO_no_update[el].lower())
    print()
    print()

NameError: name 'competention_inters' is not defined

In [16]:
for el in competention_lid:
    print(res_lid_no_update[el])
    print()

NameError: name 'competention_lid' is not defined

In [17]:
ппрпорп

NameError: name 'ппрпорп' is not defined

In [18]:
# списки только лидерских и только костромских компетенций оказались слишком большими для ручной обработки,
# поэтому для сокращения количества данных были выделены компетенции с ключевыми словами (слова, которые встречаются наиболее часто)

In [19]:
obshch_comp_text = []
lid_comp_text = []
KO_comp_text = []

for key, value in competention_inters.items():
    obshch_comp_text.append(res_lid_no_update[key])
    for el in value:
        obshch_comp_text.append(res_KO_no_update[el])

for el in competention_lid:
    lid_comp_text.append(res_lid_no_update[el])

for el in competention_KO:
    KO_comp_text.append(res_KO_no_update[el])

NameError: name 'competention_inters' is not defined

In [20]:
obshch_comp_text = list(set(obshch_comp_text))
lid_comp_text = list(set(lid_comp_text))
KO_comp_text = list(set(KO_comp_text))

In [21]:
obshch_analyz, chast_word_obshch_comp = AnalyText(obshch_comp_text)
lid_analyz, chast_word_lid_comp = AnalyText(lid_comp_text)
KO_analyz, chast_word_KO_comp = AnalyText(KO_comp_text)

NameError: name 'pymorphy2' is not defined

In [22]:
chast_word_obshch_comp

NameError: name 'chast_word_obshch_comp' is not defined

In [23]:
chast_word_lid_comp

NameError: name 'chast_word_lid_comp' is not defined

In [24]:
chast_word_KO_comp

NameError: name 'chast_word_KO_comp' is not defined

In [25]:
lid_minus_KO = set.difference(set(chast_word_lid_comp), set(chast_word_KO_comp))
pprint.pprint(lid_minus_KO)
lid_minus_KO = set.difference(set(lid_minus_KO), set(chast_word_obshch_comp))
lid_minus_KO

NameError: name 'chast_word_lid_comp' is not defined

In [26]:
lid_minus_KO = ['лазерный', 'оператор', 'программа', 'чпу']
num_comp = {'лазерный': [], 'оператор': [], 'программа': [], 'чпу': []}
for word in lid_minus_KO:
    for num, list_word in enumerate(lid_analyz):
        if word in list_word:
            num_comp[word].append(num)
num_comp

NameError: name 'lid_analyz' is not defined

In [27]:
for key, value in num_comp.items():
    print(key.upper())
    for el in value:
        print(lid_comp_text[el])
    print()

ЛАЗЕРНЫЙ

ОПЕРАТОР

ПРОГРАММА

ЧПУ



In [28]:
KO_minus_lid = set.difference(set(chast_word_KO_comp), set(chast_word_lid_comp))
print(KO_minus_lid)
KO_minus_lid = set.difference(set(KO_minus_lid), set(chast_word_obshch_comp))
print(KO_minus_lid)

NameError: name 'chast_word_KO_comp' is not defined

In [29]:
KO_minus_lid = [ 'пульт','механизм','подналадка','измерительный', 'инструкция', 'пользоваться', 'профильный']
num_comp = {'пульт': [],'механизм': [],'подналадка': [],'измерительный': [], 'инструкция': [], 'пользоваться': [], 'профильный': []}
for word in KO_minus_lid:
    for num, list_word in enumerate(KO_analyz):
        if word in list_word:
            num_comp[word].append(num)
num_comp

NameError: name 'KO_analyz' is not defined

In [30]:
for key, value in num_comp.items():
    print(key.upper())
    for el in value:
        print(KO_comp_text[el])
    print()

ПУЛЬТ

МЕХАНИЗМ

ПОДНАЛАДКА

ИЗМЕРИТЕЛЬНЫЙ

ИНСТРУКЦИЯ

ПОЛЬЗОВАТЬСЯ

ПРОФИЛЬНЫЙ

